In [ ]:
pip install catboost 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.7 MB/s 


In [ ]:
import pandas as pd 

data = pd.read_csv('facebook_chat_4a.csv', sep = ',')

facebook_chat = data[['Time', 'Protocol', 'Length']]
facebook_chat = facebook_chat.join(pd.get_dummies(facebook_chat.Protocol)).drop(['Protocol'], axis = 1)

facebook_chat['Time_diff'] = facebook_chat['Time'].diff(1)
facebook_chat = facebook_chat.iloc[1:].reset_index(drop = True)
facebook_chat = facebook_chat.drop(['Time'], axis = 1)

facebook_chat['Label'] = 0
facebook_chat.head()

,Length,ARP,DNS,GQUIC,LLMNR,NBNS,NTP,TCP,TLSv1.2,UDP,Time_diff,Label
0,66,0,0,0,0,0,0,1,0,0,0.042037,0
1,79,0,1,0,0,0,0,0,0,0,2.783592,0
2,415,0,1,0,0,0,0,0,0,0,0.000731,0
3,66,0,0,0,0,0,0,1,0,0,0.001056,0
4,92,0,0,0,0,1,0,0,0,0,0.000445,0


In [ ]:
data = pd.read_csv('facebook_audio.csv', sep = ',')

facebook_audio = data[['Time', 'Protocol', 'Length']]
facebook_audio = facebook_audio.join(pd.get_dummies(facebook_audio.Protocol)).drop(['Protocol'], axis = 1)

facebook_audio['Time_diff'] = facebook_audio['Time'].diff(1)
facebook_audio = facebook_audio.iloc[1:].reset_index(drop = True)
facebook_audio = facebook_audio.drop(['Time'], axis = 1)

facebook_audio['Label'] = 1
facebook_audio.head()

,Length,ARP,BROWSER,DNS,DTLSv1.0,GQUIC,ICMP,IGMPv3,LANMAN,NBNS,...,SMB2,SSDP,SSL,STUN,TCP,TLSv1,TLSv1.2,UDP,Time_diff,Label
0,76,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000018,1
1,76,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000004,1
2,76,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000013,1
3,1077,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.000003,1
4,583,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.000030,1


In [ ]:
# соединяем данные, чтобы добавить недостающие столбцы 

facebook = pd.concat([facebook_chat, facebook_audio], axis = 0).reset_index(drop = True)
facebook = facebook.fillna(0)
facebook.to_csv('Facebook.csv', index = False)
# 2191

In [ ]:
# разбиваем данные обратно 

facebook_chat = facebook[:2191]
facebook_audio = facebook[2191:]

In [ ]:
facebook_chat_answer = facebook_chat['Label']
facebook_chat = facebook_chat.drop(['Label'], axis = 1)

facebook_audio_answer = facebook_audio['Label']
facebook_audio = facebook_audio.drop(['Label'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(facebook_audio, facebook_audio_answer, test_size = 0.4, random_state = 0)

x_train = pd.concat([x_train, facebook_chat], axis = 0).reset_index(drop = True)
x_test = pd.concat([x_test[:len(facebook_chat_answer)], facebook_chat], axis = 0).reset_index(drop = True)

y_train = pd.concat([y_train, facebook_chat_answer], axis = 0).reset_index(drop = True)
y_test = pd.concat([y_test[:len(facebook_chat_answer)], facebook_chat_answer], axis = 0).reset_index(drop = True)

# перемешиваем тестовые данные 
mass = x_test.join(y_test)
mass_sample = mass.sample(frac = 1)

y_test = mass_sample['Label']
x_test = mass_sample.drop(['Label'], axis = 1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import mean_absolute_error, mean_squared_error

for neighbor in range(1, 11):
  knn = KNeighborsClassifier(n_neighbors = neighbor)
  knn.fit(x_train, y_train)

  knn_predict = knn.predict(x_test)

  mae = mean_absolute_error(y_test, knn_predict)
  mse = mean_squared_error(y_test, knn_predict)
  rmse = mean_squared_error(y_test, knn_predict, squared = False)

  print(neighbor, 'neighbors --> ', accuracy_score(knn_predict, y_test))
  print('       MAE', mae)
  print('       MSE', mse)
  print('       RMSE', rmse)

1 neighbors -->  0.9883614787768142
       MAE 0.01163852122318576
       MSE 0.01163852122318576
       RMSE 0.10788197821316478
2 neighbors -->  0.9922409858512095
       MAE 0.007759014148790507
       MSE 0.007759014148790507
       RMSE 0.08808526635476847
3 neighbors -->  0.9776357827476039
       MAE 0.022364217252396165
       MSE 0.022364217252396165
       RMSE 0.1495467059229195
4 neighbors -->  0.9865358283888636
       MAE 0.013464171611136467
       MSE 0.013464171611136467
       RMSE 0.1160352171159104
5 neighbors -->  0.9760383386581469
       MAE 0.023961661341853034
       MSE 0.023961661341853034
       RMSE 0.154795546905759
6 neighbors -->  0.978776814240073
       MAE 0.021223185759926974
       MSE 0.021223185759926974
       RMSE 0.14568179625446337
7 neighbors -->  0.9712460063897763
       MAE 0.02875399361022364
       MSE 0.02875399361022364
       RMSE 0.16957002568326643
8 neighbors -->  0.9744408945686901
       MAE 0.025559105431309903
       MSE 0.0255

НАИВНЫЙ БАЙЕСОВСКИЙ КЛАССИФИКАТОР *









In [ ]:
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import accuracy_score 

gnb = GaussianNB()
gnb_predict = gnb.fit(x_train, y_train).predict(x_test)

accuracy_score(gnb_predict, y_test)

0.9879050661798265

In [ ]:
# оценка точности и полноты 

tp = 0  # истино-положительное рещение 
tn = 0  # истино-отрицательное решение 
fp = 0  # ложно-положительное решение 
fn = 0  # ложно-отрицательное решение 

for i in range(len(y_test)): 
  if knn_predict[i] == y_test[i]:
    if y_test[i] == 1: tp += 1
    else: tn += 1 
  else: 
    if y_test[i] == 0 and knn_predict[i] == 1: fp += 1
    else: fn += 1

print('для knn_predict')
print('точность', tp / (tp + fp))
print('полнота', tp / (tp + fn))


# для gnb_predict
# точность 0.4939365671641791
# полнота 0.4833409402099498

МЕТОД ОПОРНЫХ ВЕКТОРОВ 

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score 

clf = svm.SVC()
clf_predict = clf.fit(x_train, y_train).predict(x_test)

accuracy_score(clf_predict, y_test)

0.8224554997717937

 МЕТОД ГРАДИЕНТНОГО СПУСКА 

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score 

clf = SGDClassifier(loss = "hinge", penalty = "l2", max_iter = 5)
clf_predict = clf.fit(x_train, y_train).predict(x_test)

accuracy_score(clf_predict, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


0.5

[ метод опорных векторов и метод градиентного спуска показывают более низкий результат с уменьшением тестовых данных ]

проблемы: 
- в самом маленьком файле 145 строк -> неравномерное распределение данных, что повлияет на работу модели 